In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import warnings
# filter some warning messages
warnings.filterwarnings("ignore") 

####################you will need to change some paths here!#####################
#list of input files
filename_origin='f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info.nc'
# output files
filename_origin_out='f:/data/project_data/NASA_biophysical/collocated_data/NPPSD_GOA_allseabird_full_eddy_info_envdata.nc'


In [2]:
ds_bird = xr.open_dataset(filename_origin)
ds_bird

<xarray.Dataset>
Dimensions:             (observation_number: 1830, track: 765)
Coordinates:
  * track               (track) int32 4438 4918 5213 ... 270829 270892 271234
  * observation_number  (observation_number) int32 0 1 2 3 ... 1827 1828 1829
Data variables:
    amplitude           (track, observation_number) float32 ...
    cyclonic_type       (track, observation_number) float64 ...
    lat                 (track, observation_number) float32 ...
    lon                 (track, observation_number) float32 ...
    speed_average       (track, observation_number) float32 ...
    speed_radius        (track, observation_number) float32 ...
Attributes:
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    title:                     Mesoscale Eddies in Altimeter Observations of SLA
    institution:               CNES/CLS in collaboration with Oregon State Un...
    project:                   SSALTO/DUACS
    creator_url:               http://www.aviso.altimetry.fr
    creator_email:             aviso@altimetry.fr
    license:                   http://www.aviso.altimetry.fr/fileadmin/docume...
    summary:                   This dataset contains eddy atlas from two-sate...
    comment:                   Surface product; mesoscale eddies
    input_product_reference:   https://www.aviso.altimetry.fr/en/data/product...
    date_created:              2017-09-22T10:20:17Z
    time_coverage_duration:    P8771D
    time_coverage_start:       1993-01-01T00:00:00Z
    time_coverage_end:         2017-01-06T00:00:00Z
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    product_version:           1.0

In [3]:
def get_data():
    
    #climatology years
    cyr1,cyr2='1993-01-01','2018-12-31'
    
    # CCMP test
    dir_pattern_zarr = 'F:/data/sat_data/ccmp/zarr/'
    ds= xr.open_zarr(dir_pattern_zarr)
    ds = ds.rename({'latitude':'lat','longitude':'lon'})
    ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
    ds_ccmp = ds.sortby(ds.lon)
    ds_ccmp = ds_ccmp.drop('nobs')
    for var in ds_ccmp:
        tem = ds_ccmp[var].attrs
        tem['var_name']='ccmp_'+str(var)
        ds_ccmp[var].attrs=tem
    ds_ccmp_clim = ds_ccmp.sel(time=slice(cyr1,cyr2))
    ds_ccmp_clim = ds_ccmp_clim.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
    
    # AVISO test
    dir_pattern_zarr = 'F:/data/sat_data/aviso/zarr/'
    ds= xr.open_zarr(dir_pattern_zarr)
    ds = ds.rename({'latitude':'lat','longitude':'lon'})
    ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
    ds_aviso = ds.sortby(ds.lon).drop({'lat_bnds','lon_bnds','crs','err'})
    for var in ds_aviso:
        tem = ds_aviso[var].attrs
        tem['var_name']='aviso_'+str(var)
        ds_aviso[var].attrs=tem
    ds_aviso_clim = ds_aviso.sel(time=slice(cyr1,cyr2))
    ds_aviso_clim = ds_aviso_clim.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)    

    #sst
    dir_pattern_zarr = 'F:/data/sat_data/sst/cmc/zarr/'
    ds_sst= xr.open_zarr(dir_pattern_zarr)
    ds_sst = ds_sst.drop({'analysis_error','mask','sea_ice_fraction'})
    tem = ds_sst.analysed_sst.attrs
    tem['var_name']='cmc_sst'
    ds_sst.analysed_sst.attrs=tem
    ds_sst_clim = ds_sst.sel(time=slice(cyr1,cyr2))
    ds_sst_clim = ds_sst_clim.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
    
    #get bathymetry from ETOPO1
    fname_topo = 'F:/data/topo/ETOPO1_Ice_g_gmt4.grd'
    ds = xr.open_dataset(fname_topo)
    ds_topo = ds.rename_dims({'x':'lon','y':'lat'}).rename({'x':'lon','y':'lat'})
    tem = ds_topo.z.attrs
    tem['var_name']='etopo_depth'
    ds_topo.z.attrs=tem

    #put data into a dictionary
    data_dict={'aviso':ds_aviso,
               'wnd':ds_ccmp,
               'sst':ds_sst,
              'topo':ds_topo}
    clim_dict={'aviso_clim':ds_aviso_clim,
               'wnd_clim':ds_ccmp_clim,
               'sst_clim':ds_sst_clim}
  
    return data_dict,clim_dict

data,clim = get_data()

In [4]:
#ds_bird = xr.open_dataset(filename_bird_out_eddy_netcdf)
ilen_bird1 = len(ds_bird.track)
ilen_bird2 = len(ds_bird.observation_number)
for name in data:
    ds_data=data[name]
    if name=='topo':
        continue
    if not name==input_data:
        continue
    print('name',name)   
    for var in ds_data:
        var_tem=ds_data[var].attrs['var_name']
        ds_bird[var_tem]=xr.DataArray(np.nan*np.empty((ilen_bird1,ilen_bird2), 
                                                      dtype=str(ds_data[var].dtype)), 
                                      coords={'track': ds_bird.track,'observation_number':'ds_bird.observation_number'},
                                      dims=('track','observation_number'))
        ds_bird[var_tem].attrs=ds_data[var].attrs
    print('var',var_tem)
    for i in range(ilen_bird1):
        for j in range(ilen_bird2):
            if np.isnan(ds_bird.lat[i,j]):
                continue
            if ds_bird.time[i,j]<ds_data.time.min():
                continue
            if ds_bird.time[i,j]>ds_data.time.max():
                continue
            t1,t2 = ds_bird.time[i,j]-np.timedelta64(24,'h'), ds_bird.time[i,j]+np.timedelta64(24,'h')
            lat1,lat2=ds_bird.lat[i,j]-.5,ds_bird.lat[i,j]+.5
            lon1,lon2=ds_bird.lon[i,j]-.5,ds_bird.lon[i,j]+.5
            tem = ds_data.sel(time=slice(t1,t2),lat=slice(lat1,lat2),lon=slice(lon1,lon2)).load()
            tem = tem.interp(time=ds_bird.time[i,j],lat=ds_bird.lat[i,j],lon=ds_bird.lon[i,j])
            for var in ds_data:
                var_tem=ds_data[var].attrs['var_name']
                ds_bird[var_tem][i,j]=tem[var].data
        print(i)
        #at topo info


NameError: name 'input_data' is not defined

In [ ]:
#interp will create a new 2D array, to avoid that put the lat/lon into dataarrays
ds_topo=data['topo']
new_lat = xr.DataArray(ds_bird.lat.data, dims='z')
new_lon = xr.DataArray(ds_bird.lon.data, dims='z')
tem = ds_topo.z.interp(lat=new_lat, lon=new_lon,method='nearest')
ds_bird['ETOPO_depth'] = xr.DataArray(tem.data, coords={'track': ds_bird.track,'observation_number':'ds_bird.observation_number'},
                                      dims=('track','observation_number'))
#ds_bird = ds_bird.drop('obs')

# collocate color

In [ ]:
ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v4.2-DAILY')
for var in ds_color:
    if not var=='chlor_a':
        ds_color = ds_color.drop(var)
ds_color

In [ ]:
ilen_bird = len(ds_bird.lat)
ds_data = ds_color
for var in ds_data:
    var_tem=var
    ds_bird[var_tem]=xr.DataArray(np.nan*np.empty(ilen_bird, dtype=str(ds_data[var].dtype)), coords={'index': ds_bird.index}, dims=('index'))
    ds_bird[var_tem].attrs=ds_data[var].attrs
print('var',var_tem)
for i in range(ilen_bird):
    t1,t2 = ds_bird.time64[i]-np.timedelta64(5,'D'), ds_bird.time64[i]+np.timedelta64(5,'D')
    lat1,lat2=ds_bird.lat[i]+.1,ds_bird.lat[i]-.1
    lon1,lon2=ds_bird.lon[i]-.1,ds_bird.lon[i]+.1
    tem = ds_data.sel(time=slice(t1,t2),lat=slice(lat1,lat2),lon=slice(lon1,lon2)).load()
    if len(tem)<1:
        continue
    tem = tem.mean(keep_attrs=True)
    for var in ds_data:
        var_tem=var
        ds_bird[var_tem][i]=tem[var].data
    if int(i/100)*100==i:
        print(i,len(ds_bird.lat))

In [ ]:
#output data
df_bird = ds_bird.to_dataframe()
df_bird.to_csv(filename_bird_out+input_data+'.csv')
ds_bird.to_netcdf(filename_bird_out_netcdf+input_data+'.nc')

